In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error,r2_score
from sklearn.linear_model import LinearRegression

In [2]:
# Load dataset
df = pd.read_csv('carvana_car_sold-2022-08.csv')
df_one = pd.read_csv('used_car_sales.csv')
#df_two = pd.read_csv('cars.csv')

In [3]:
df.isnull().sum()

vehicle_id                0
stock_number              0
year                      0
make                      0
model                     0
miles                     0
trim                      0
sold_price                0
discounted_sold_price     0
partnered_dealership      0
delivery_fee              0
earliest_delivery_date    0
sold_date                 0
dtype: int64

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44365 entries, 0 to 44364
Data columns (total 13 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   vehicle_id              44365 non-null  int64 
 1   stock_number            44365 non-null  int64 
 2   year                    44365 non-null  int64 
 3   make                    44365 non-null  object
 4   model                   44365 non-null  object
 5   miles                   44365 non-null  int64 
 6   trim                    44365 non-null  object
 7   sold_price              44365 non-null  int64 
 8   discounted_sold_price   44365 non-null  int64 
 9   partnered_dealership    44365 non-null  bool  
 10  delivery_fee            44365 non-null  int64 
 11  earliest_delivery_date  44365 non-null  object
 12  sold_date               44365 non-null  object
dtypes: bool(1), int64(7), object(5)
memory usage: 4.1+ MB


In [24]:
df

,vehicle_id,stock_number,year,make,model,miles,trim,sold_price,discounted_sold_price,partnered_dealership,delivery_fee,earliest_delivery_date,sold_date
0,2270123,2001646447,2014,Volkswagen,Jetta,53238,2.0L S,15990,15990,False,1490,2022-09-09T21:34:31.376Z,8/31/2022
1,2298228,2001690283,2015,Chevrolet,Volt,112160,base,17590,17590,False,790,2022-09-09T22:04:42.555Z,8/31/2022
2,2463098,2001933427,2015,Ford,Focus,57046,SE,14990,14990,False,0,2022-09-06T21:35:20.3145311Z,8/31/2022
3,2488480,2001977729,2013,Honda,Odyssey,103874,EX-L,18990,18990,False,490,2022-09-06T22:04:19.711928Z,8/31/2022
4,2183400,2001531442,2015,Chevrolet,Cruze,86301,1LT,13990,13990,False,790,2022-09-09T22:05:19.367Z,8/31/2022
...,...,...,...,...,...,...,...,...,...,...,...,...,...
44360,2116890,2001435635,2012,Hyundai,Accent,96252,SE,13590,13590,False,1490,2022-08-08T18:06:28.517Z,8/1/2022
44361,2267923,2001638155,2013,Lincoln,MKX,40491,base,23590,23590,False,490,2022-08-08T21:34:22.412Z,8/1/2022
44362,2321468,2001707909,2012,Honda,Civic,72228,LX,14990,14990,False,490,2022-08-08T21:38:00.139Z,8/1/2022
44363,2326171,2001718043,2011,Lexus,LS,79020,LS 460 L,26590,26590,False,190,2022-08-07T21:28:54.2596276Z,8/1/2022


In [25]:
df.select_dtypes(['int64']).corr()

,vehicle_id,stock_number,year,miles,sold_price,discounted_sold_price,delivery_fee
vehicle_id,1.000000,0.971891,0.085837,-0.039330,0.058389,0.058389,-0.081768
stock_number,0.971891,1.000000,0.082694,-0.047570,0.066438,0.066438,-0.084804
year,0.085837,0.082694,1.000000,-0.566231,0.525812,0.525812,-0.112738
miles,-0.039330,-0.047570,-0.566231,1.000000,-0.554410,-0.554410,0.078524
sold_price,0.058389,0.066438,0.525812,-0.554410,1.000000,1.000000,-0.059324
discounted_sold_price,0.058389,0.066438,0.525812,-0.554410,1.000000,1.000000,-0.059324
delivery_fee,-0.081768,-0.084804,-0.112738,0.078524,-0.059324,-0.059324,1.000000


In [26]:
# Handle missing values for 'location' column
if 'location' in df.columns:
    df['location'].fillna('Unknown', inplace=True)
else:
    print("The 'location' column is not available.")

# Handle missing values for 'state' column
if 'state' in df.columns:
    df['state'].fillna('Unknown', inplace=True)
else:
    print("The 'state' column is not available.")

The 'location' column is not available.
The 'state' column is not available.


In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44365 entries, 0 to 44364
Data columns (total 13 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   vehicle_id              44365 non-null  int64 
 1   stock_number            44365 non-null  int64 
 2   year                    44365 non-null  int64 
 3   make                    44365 non-null  object
 4   model                   44365 non-null  object
 5   miles                   44365 non-null  int64 
 6   trim                    44365 non-null  object
 7   sold_price              44365 non-null  int64 
 8   discounted_sold_price   44365 non-null  int64 
 9   partnered_dealership    44365 non-null  bool  
 10  delivery_fee            44365 non-null  int64 
 11  earliest_delivery_date  44365 non-null  object
 12  sold_date               44365 non-null  object
dtypes: bool(1), int64(7), object(5)
memory usage: 4.1+ MB


In [28]:
# Features to use for training
features = ['year','make','model', 'miles', 'trim']

In [29]:
# Target variable
target = 'sold_price'

In [30]:
# Separating features and target
x = df[features]
y = df[target]

In [31]:
# Splitting into train and test sets
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [32]:
x.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44365 entries, 0 to 44364
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   year    44365 non-null  int64 
 1   make    44365 non-null  object
 2   model   44365 non-null  object
 3   miles   44365 non-null  int64 
 4   trim    44365 non-null  object
dtypes: int64(2), object(3)
memory usage: 1.7+ MB


In [33]:
# Preprocessing pipeline for numerical and categorical data
numerical_features = ['year', 'miles']
categorical_features = ['make','model','trim']


In [34]:
# Preprocessing pipeline
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ])



In [35]:

# Model pipeline
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', RandomForestRegressor(random_state=42))
])

In [36]:

# Training the model
model.fit(x_train, y_train)


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  ['year', 'miles']),
                                                 ('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['make', 'model', 'trim'])])),
                ('regressor', RandomForestRegressor(random_state=42))])

In [37]:
# Predictions
y_pred = model.predict(x_test)

In [38]:
# Evaluation
r2 =r2_score(y_test,y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)

In [39]:
print(r2)
print(rmse)

0.906884977843628
3170.7043657510094


In [40]:
print(f'Root Mean Squared Error (RMSE): {rmse}')

Root Mean Squared Error (RMSE): 3170.7043657510094


In [41]:
numerical_features = ['year', 'miles']
categorical_features = ['make','model','trim']
# Features to use for training
features = ['year','make','model', 'miles', 'trim']

In [42]:
# Function to get user input and make predictions
def get_user_input_and_predict():
    # Getting user inputs for each feature
    year = int(input("Enter the year of the car (e.g., 2021): "))
    make = input("Enter the car make (e.g., ms, mx, my): ")
    model  = input("Enter the car model (e.g., ms, mx, my): ")
    miles = int(input("Enter the mileage of the car: "))
    trim = input("Enter the trim (e.g., Long Range All-Wheel Drive): ")

    # Creating a dataframe for the user input
    user_data = pd.DataFrame({
        'year': [year],
        "make": [make],
        'model': [model],
        'miles': [miles],
        'trim': [trim]
    })
    return user_data


In [43]:
df

,vehicle_id,stock_number,year,make,model,miles,trim,sold_price,discounted_sold_price,partnered_dealership,delivery_fee,earliest_delivery_date,sold_date
0,2270123,2001646447,2014,Volkswagen,Jetta,53238,2.0L S,15990,15990,False,1490,2022-09-09T21:34:31.376Z,8/31/2022
1,2298228,2001690283,2015,Chevrolet,Volt,112160,base,17590,17590,False,790,2022-09-09T22:04:42.555Z,8/31/2022
2,2463098,2001933427,2015,Ford,Focus,57046,SE,14990,14990,False,0,2022-09-06T21:35:20.3145311Z,8/31/2022
3,2488480,2001977729,2013,Honda,Odyssey,103874,EX-L,18990,18990,False,490,2022-09-06T22:04:19.711928Z,8/31/2022
4,2183400,2001531442,2015,Chevrolet,Cruze,86301,1LT,13990,13990,False,790,2022-09-09T22:05:19.367Z,8/31/2022
...,...,...,...,...,...,...,...,...,...,...,...,...,...
44360,2116890,2001435635,2012,Hyundai,Accent,96252,SE,13590,13590,False,1490,2022-08-08T18:06:28.517Z,8/1/2022
44361,2267923,2001638155,2013,Lincoln,MKX,40491,base,23590,23590,False,490,2022-08-08T21:34:22.412Z,8/1/2022
44362,2321468,2001707909,2012,Honda,Civic,72228,LX,14990,14990,False,490,2022-08-08T21:38:00.139Z,8/1/2022
44363,2326171,2001718043,2011,Lexus,LS,79020,LS 460 L,26590,26590,False,190,2022-08-07T21:28:54.2596276Z,8/1/2022


In [44]:
# Example usage in a notebook cell:
data = get_user_input_and_predict()
model.predict(data)

array([31098.2])

In [45]:
import joblib
joblib.dump(model,'model.joblib')

['model.joblib']